In [12]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [27]:
#importing test and train
df_train=pd.read_csv('train.csv')
#df_train.shape
df_test=pd.read_csv('test.csv')
#df_test.head()
df_train['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [28]:
#combining test and train
df=pd.concat([df_train,df_test],axis=0)
Y=df.iloc[:1460,:81]
Y=Y.iloc[:,-1]
df.drop(['SalePrice'],axis=1,inplace=True)
df.shape

(2919, 80)

In [36]:
#finding nan values
for i in df.columns.tolist():
    print(i,'\t',df[i].isnull().sum())

Id 	 0
MSSubClass 	 0
MSZoning 	 0
LotFrontage 	 0
LotArea 	 0
Street 	 0
Alley 	 0
LotShape 	 0
LandContour 	 0
Utilities 	 0
LotConfig 	 0
LandSlope 	 0
Neighborhood 	 0
Condition1 	 0
Condition2 	 0
BldgType 	 0
HouseStyle 	 0
OverallQual 	 0
OverallCond 	 0
YearBuilt 	 0
YearRemodAdd 	 0
RoofStyle 	 0
RoofMatl 	 0
Exterior1st 	 0
Exterior2nd 	 0
MasVnrType 	 0
MasVnrArea 	 0
ExterQual 	 0
ExterCond 	 0
Foundation 	 0
BsmtQual 	 0
BsmtCond 	 0
BsmtExposure 	 0
BsmtFinType1 	 0
BsmtFinSF1 	 0
BsmtFinType2 	 0
BsmtFinSF2 	 0
BsmtUnfSF 	 0
TotalBsmtSF 	 0
Heating 	 0
HeatingQC 	 0
CentralAir 	 0
Electrical 	 0
1stFlrSF 	 0
2ndFlrSF 	 0
LowQualFinSF 	 0
GrLivArea 	 0
BsmtFullBath 	 0
BsmtHalfBath 	 0
FullBath 	 0
HalfBath 	 0
BedroomAbvGr 	 0
KitchenAbvGr 	 0
KitchenQual 	 0
TotRmsAbvGrd 	 0
Functional 	 0
Fireplaces 	 0
FireplaceQu 	 0
GarageType 	 0
GarageYrBlt 	 0
GarageFinish 	 0
GarageCars 	 0
GarageArea 	 0
GarageQual 	 0
GarageCond 	 0
PavedDrive 	 0
WoodDeckSF 	 0
OpenPorchSF 	 

In [30]:
#list of features to drop
l=[]
for i in df.columns.tolist():
    if (df[i].isnull().sum()>1400):
        l.append(i)
#del[l[len(l)-1]]
print(l)

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']


In [31]:
#drop feature function
def drop_shit(dff,l):
    for i in l:
        dff.drop([i],axis=1,inplace=True)
    return dff

df.shape


(2919, 80)

In [32]:
#dealing with remaining null values
r=[]
for i in df.columns.tolist():
    if df[i].isnull().sum()>0:
        r.append(i)


In [33]:
q=list()
w=list()
#list till now:l,r,q,w
for i in r:
    if df[i].dtype == df['MSZoning'].dtype:
        q.append(i)
    else:
        w.append(i)

In [34]:
#fill nan of integer and object type
def fill_objectnan(dff,q):
    for i in q:
        dff[i].fillna(dff[i].mode()[0],inplace=True)
    return dff
def fill_intnan(dff,w):
    for i in w:
        dff[i].fillna(dff[i].mean(),inplace=True)
    return dff    
df=fill_objectnan(df,q)
df=fill_intnan(df,w)
        

In [35]:
#handling categorical data
#list till now:l,r,q,w,e
e=list()
for i in df.columns.tolist():
    if df[i].dtype== df['MSZoning'].dtype:
        e.append(i)       

In [37]:
#for more than 5 values use labelencoder else use onehotencoder
from sklearn.preprocessing import LabelEncoder
def encode_categorical(dff,e):
    for i in e:
        if len(df[i].value_counts())<=5:
            d=pd.get_dummies(dff[i],drop_first=True)
            dff.drop([i],axis=1,inplace=True)
            dff=pd.concat([dff,d],axis=1)
        else:
            le=LabelEncoder()
            dff[i]=le.fit_transform(dff[i])
    return dff
df=encode_categorical(df,e)

In [38]:
#dropping duplicate columns
df =df.loc[:,~df.columns.duplicated()]

In [53]:
df_train=df.iloc[:1460,:]
df_test=df.iloc[1460:,:]
df_test.head()

,Id,MSSubClass,LotFrontage,LotArea,Neighborhood,Condition1,Condition2,HouseStyle,OverallQual,OverallCond,...,SBrkr,RFn,Unf,P,GdWo,MnPrv,MnWw,Othr,Shed,TenC
0,1461,20,80.0,11622,12,1,2,2,5,6,...,1,0,1,0,0,1,0,0,1,0
1,1462,20,81.0,14267,12,2,2,2,6,6,...,1,0,1,0,0,1,0,0,0,0
2,1463,60,74.0,13830,8,2,2,5,5,5,...,1,0,0,0,0,1,0,0,1,0
3,1464,60,78.0,9978,8,2,2,5,6,6,...,1,0,0,0,0,1,0,0,1,0
4,1465,120,43.0,5005,22,2,2,2,8,5,...,1,1,0,0,0,1,0,0,1,0


In [57]:
#FEATURE SELECTIONdeleting unnecessary features
from sklearn.feature_selection import chi2,SelectKBest
select=SelectKBest(score_func=chi2,k=10)
fit=select.fit(df_train,Y)
dfscores=pd.DataFrame(fit.scores_)
dfcolumns=pd.DataFrame(df_train.columns)
Scores=pd.concat([dfcolumns,dfscores],axis=1)
Scores.columns=['Specs','Score']
delete=Scores.nsmallest(5,'Score')
delete
#delete unnecessary columns accordingly then split and check the shape is correct in below columns

,Specs,Score
50,YrSold,0.602971
96,Shed,1.857829
57,Pave,3.650601
6,Condition2,27.336481
34,KitchenAbvGr,28.490834


In [59]:
#delete in df_test accordingly:
for i in delete['Specs']:
    df_train.drop([i],axis=1,inplace=True)
    df_test.drop([i],axis=1,inplace=True)

C:\Users\aksha\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [60]:
print(df_train.shape)
print(df_test.shape)
print(Y.shape)
'''
(1460, 98)
(1459, 98)
(1460,)
'''

(1460, 93)
(1459, 93)
(1460,)


'\n(1460, 98)\n(1459, 98)\n(1460,)\n'

In [68]:
#xgboost with hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV
import xgboost
params={
    'learning_rate':[0.05,0.10,0.15,0.20,0.25,0.30,0.35],
    'max_depth':[3,4,5,6,7,8,9,10,12,15],
    'min_child_weight':[1,3,5,7],
    'gamma':[0.1,0.2,0.3,0.4,0.5],
    'colsample_bytree':[0.3,0.4,0.5,0.7]
}
model=xgboost.XGBRegressor()
random=RandomizedSearchCV(model,param_distributions=params,n_iter=5,scoring='neg_root_mean_squared_error',cv=5,verbose=3)
random.fit(df_train,Y)
random.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4 
[CV]  min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4, score=-24408.801, total=   0.2s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



[CV] min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4 
[CV]  min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4, score=-32489.435, total=   0.1s
[CV] min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV]  min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4, score=-28759.816, total=   0.2s
[CV] min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4 
[CV]  min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4, score=-23311.824, total=   0.2s
[CV] min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4 
[CV]  min_child_weight=1, max_depth=5, learning_rate=0.25, gamma=0.5, colsample_bytree=0.4, score=-27513.689, total=   0.2s
[CV] min_child_weight=1, max_depth=12, learning_rate=0.3, gamma=0.3, colsample_bytree=0.7 
[CV]  min_child_weight=1, max_depth=12, learning_rate=0.3, gamma=0.3, colsample_bytree=0.7, score=-28099.989, total=   0.4s
[CV] min_child_weight=1, max_depth=12, learning_rate=0.3, gamma=0.3, colsample_bytree=0.7 
[CV]  min_child_weight=1, max_depth=12, learning_rate=0.3, gamma=0.3, colsample_bytree=0.7, score=-36616.717, total=   0.4s
[CV] min_child_w

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    7.7s finished


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=12,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [71]:
#implementing xgboost regressor with hypertuned methods
model_xgb=xgboost.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=12,
             min_child_weight=5, missing=np.nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
model_xgb.fit(df_train,Y)
predict_xgb=model_xgb.predict(df_test)

In [81]:
#saving the predictions to a dataframe
sample=pd.read_csv('sample_submission.csv')
final_xgb=pd.DataFrame({'Id':sample['Id'],
                       'SalePrice':predict_xgb})

In [83]:
final_xgb.to_csv("answer_xgb.csv",index=False)

In [86]:
#using neural networks with hyperparameter tuning:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [97]:
regressor=Sequential()
regressor.add(Dense(units = 50,activation='relu',input_dim = 93))
regressor.add(Dense(units = 25,activation='relu'))
regressor.add(Dense(units = 50, activation='relu'))
regressor.add(Dense(units = 1))
regressor.compile(loss=root_mean_squared_error,optimizer='Adamax')
#model_history=regressor.fit(df_train.values, Y.values,validation_split=0.20, batch_size = 10, epochs = 1000)

In [89]:
from tensorflow.keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [122]:
pred_ann=regressor.predict(df_test.values)

In [125]:
pred=pd.DataFrame(pred_ann)
final_ann=pd.concat([sample['Id'],pred],axis=1)
final_ann.columns=['Id','Saleprice']
final_ann.to_csv("answer_ann.csv",index=False)